In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
# for console formatting I use the Rich library
from rich.console import Console

console = Console()

In [3]:
def get_llm_client(llm_choice):
    if llm_choice == "GROQ":
        client = OpenAI(
            base_url="https://api.groq.com/openai/v1",
            api_key=os.environ.get("GROQ_API_KEY"),
        )
        return client
    elif llm_choice == "OPENAI":
        load_dotenv()  # load environment variables from .env fil
        client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
        return client
    else:
        raise ValueError("Invalid LLM choice. Please choose 'GROQ' or 'OPENAI'.")

In [4]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

LLM_CHOICE = "OPENAI"
LLM_CHOICE = "GROQ"

if OPENAI_API_KEY:
    print(f"OPENAI_API_KEY exists and begins {OPENAI_API_KEY[:14]}...")
else:
    print("OPENAI_API_KEY not set")

if GROQ_API_KEY:
    print(f"GROQ_API_KEY exists and begins {GROQ_API_KEY[:14]}...")
else:
    print("GROQ_API_KEY not set")


client = get_llm_client(LLM_CHOICE)
if LLM_CHOICE == "GROQ":
    MODEL = "llama-3.3-70b-versatile"
else:
    MODEL = "gpt-4o-mini"

print(f"LLM_CHOICE: {LLM_CHOICE} - MODEL: {MODEL}")

OPENAI_API_KEY exists and begins sk-proj-vIt-L1...
GROQ_API_KEY exists and begins gsk_0yKDCuUXkz...
LLM_CHOICE: GROQ - MODEL: llama-3.3-70b-versatile


Essentially, we build up data to pass as SYSTEM MESSAGE that sets the context for the conversation and contexctual data for the LLM.

I split this into core SYSTEM_MESSAGE and PROMPT_ENGINEERING = system_message

We then create the USER PROMPT that is the input to the LLM.

OpenAI was trained on this structure, so it is most effective to do it this way.


In [5]:
# We now create our ENDPOINT but on the CLIENT side and in NATURAL LANGUAGE

system_message = """
You are an assistant that is great at telling jokes.
"""

# Here is where we can do some prompt engineering - we are adding to the system message and creating our endpoint as it were.

prompt_engineering = """
01_demo_api.ipynb

# Background

You are an assistant that is great at telling jokes.

# Objective

To return a joke worthy of publishing including a rating out of 10 an whether to PUBLISH or RETRY as the *next* step.

# Suitable to publish

A joke is worthy of being published if it has a score of 8.5/10 or higher and is deemed suitable to be published by the editor.

# Example Output

system_message = 'You are an assistant that is great at telling jokes.'

prompt_engineering = 'A joke worthy of publishing is a joke that you feel is OK or higher and above by your own standards, with the following rating scale in ascending order of quality - POOR, OK, MODERATELY GOOD, GOOD, VERY GOOD, EXCELLENT'

If the joke is worthy of publishing also include next: PUBLISH otherwise next: RETRY
# Example


Here is an example of a joke worthy of publishing:

Supply the response in the following JSON format:

{"setup": "The setup of the joke",
"punchline": "The punchline of the joke",   
"rating": "GOOD",
"next": "PUBLISH",
"explanation": "This joke is funny beacuse it plays on the idea of a common phrase and gives it a humorous twist."}

Remove all back ticks and other unnecessary characters and just print the JSON format and nothing else.

Always give a new joke each time.

**Give a short explanation of the joke in a separate JSON object with the key "explanation" and the value being the explanation.**

Remove all back ticks and other unnecessary characters and just print the JSON format and nothing else.

Please ensure jokes are not repeated on retries
"""
# We add the prompt engineering to the system message
system_message += prompt_engineering


# We create our user prompt and will add all of these to the payload.
user_prompt = "Tell a light-hearted joke for a general audience"

In [6]:
# We create a list of messages to pass to the LLM

# OpenAI works was trained with a list of messages - system, user, assistant - so using this is most effective

prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt},
]
# we will get back as we have seen previously an 'assistant' message.

In [ ]:
# Standard API request
response = client.chat.completions.create(
    model=MODEL, messages=prompts, temperature=0.7
)
print(response.choices[0].message.content)  # location of answer
output = response.choices[0].message.content.replace("\n", "")

{"setup": "Why couldn't the bicycle stand up by itself?", 
"punchline": "Because it was two-tired", 
"rating": 9, 
"next": "PUBLISH", 
"explanation": "This joke is funny because it plays on the common phrase 'two-tired' meaning exhausted, but also references the fact that a bicycle has two tires."}


In [17]:
output = dict(json.loads(output))
output

{'setup': "Why couldn't the bicycle stand up by itself?",
 'punchline': 'Because it was two-tired',
 'rating': 9,
 'next': 'PUBLISH',
 'explanation': "This joke is funny because it plays on the common phrase 'two-tired' meaning exhausted, but also references the fact that a bicycle has two tires."}

### Autonomy - NEXT determined by Agent

This can be used to determine the next step to take in an 'if/else' type scenario


In [ ]:
console.print(f"[cyan bold]NEXT: {output["next"]}")  # publish or retry?

NEXT: PUBLISH